## Imports

In [1]:
import tensorflow as tf
from keras import layers, models
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os
# from time import sleep
import cv2
# import pandas as pd
# import logging

2023-03-05 13:34:48.167402: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 13:34:48.278728: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-05 13:34:48.278755: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-05 13:34:48.807498: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
sess = tf.compat.v1.Session(config=config)

2023-03-04 15:53:19.220986: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 15:53:19.307875: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jarvis/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-03-04 15:53:19.308343: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-04 15:53:19.308735: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (training-server): /proc/driver/nvidia/version

In [ ]:
# initalize logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', filename='main_log_v1.log', filemode='w')
# logging.basicConfig(level=logging.ERROR, 
#                     format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', 
#                     datefmt='%d-%b-%y %H:%M:%S', 
#                     filename='main_log_v1.log', 
#                     filemode='w')


In [7]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")


Please install GPU version of TF


In [6]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU and enable memory growth
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        # print(len(gpus), "Physical GPUs:", len(logical_gpus), "Logical GPU")
        print(f'Physical GPUs: {gpus} Logical GPUs: {logical_gpus}')
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print('Error: ', e)
else:
    print('No GPUs found')


No GPUs found


## Making the dataset

In [ ]:
# Capture video from webcam and make dataset
class MakeDataset():
    def __init__(self):
        import cv2
        import os
        
        self.video = cv2.VideoCapture(0)
        self.count = 0
        self.person_name = input("Enter person name: ")
        # self.num_images = int(input("Enter number of images: "))
        self.path = "data/" + self.person_name
        if not os.path.exists(self.path):
            os.makedirs(self.path)
    def make(self):
        while True:
            _, frame = self.video.read()
            cv2.imshow("frame", frame)
            if cv2.waitKey(1) & 0xFF == ord('s'):
                self.count += 1
                cv2.imwrite(self.path + "/" + self.person_name + '_' + str(self.count) + ".jpg", frame)
                print(f'Image saved: {self.count}', end='\r')
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            if self.count == 50:
                break
    def __final__(self):
        self.video.release()
        cv2.destroyAllWindows()

# MakeDataset().make()

## Loading the data

### 1. Listing images in the dataset directory

In [ ]:
# # Devasheesh's faces
# deva_faces = './data/devasheesh/'
# deva_faces = os.listdir(deva_faces)
# deva_faces = [deva_faces[i] for i in range(len(deva_faces)) if deva_faces[i].endswith('.jpg')]

# # Swarnim's faces
# swar_faces = './data/swarnim/'
# swar_faces = os.listdir(swar_faces)
# swar_faces = [swar_faces[i] for i in range(len(swar_faces)) if swar_faces[i].endswith('.jpg')]

# # Negative faces
# neg_faces = './data/negative-faces/'
# neg_faces = os.listdir(neg_faces)
# neg_faces = [neg_faces[i] for i in range(len(neg_faces)) if neg_faces[i].endswith('.jpg')]

# Making a function to list all the images in a folder
def list_images(path):
    image_files = []
    count = 0
    for root, _, files in os.walk(path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_files.append(os.path.join(root, file))
                print(f'Image found: {count}', end='\r')
                count += 1
    image_files.sort()
    return image_files
neg_faces = list_images('/coding-drive/DATASETS/negative-faces/VGG-Face2/data/test/')
deva_faces = list_images('/coding-drive/DATASETS/positive_face_datase/Devasheesh/')
swar_faces = list_images('/coding-drive/DATASETS/positive_face_datase/Swarnim/')

In [ ]:
neg_faces = list_images('/coding-drive/DATASETS/negative-faces/VGG-Face2/data/test/')
# neg_faces = list_images(r'Y:\DATASETS\negative-faces\VGG-Face2\data\test')


In [ ]:
deva_faces = list_images('/coding-drive/DATASETS/positive_face_datase/Devasheesh/')
# deva_faces = list_images(r'Y:\DATASETS\positive_face_dataset\Devasheesh')
# deva_faces

In [ ]:
swar_faces = list_images('/coding-drive/DATASETS/positive_face_datase/Swarnim/')
# swar_faces = list_images(r'Y:\DATASETS\positive_face_dataset\Swarnim')

### 2. Making Train and Test Objects

In [ ]:
# function to extract the face from the image
# using the Haar Cascade Classifier
def extract_frontal_face_harr(image_ndarray, grayscale=True, size=(150, 150)):
    # Load the cascade
    face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
    # Convert into grayscale
    if grayscale:
        image_gray = cv2.cvtColor(image_ndarray, cv2.COLOR_BGR2GRAY)
    else:
        image_gray = image_ndarray
    # Detect faces
    faces_cord = face_cascade.detectMultiScale(image_gray, 1.3, 5)
    # Return the face or None if not found
    if len(faces_cord) == 0:
        return None, None
    # Extract the face
    (x, y, w, h) = faces_cord[0]
    # Resize the image to 150x150
    image_gray_resized = cv2.resize(image_gray[y:y+w, x:x+h], size)
    # Return only the face part of the image
    return image_gray_resized, faces_cord

# using DNN
modelFile = "dnn/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "dnn/deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

def extract_frontal_face(image_ndarray, grayscale=False, size=(300, 300)):
    # resize the image to 300x300 for the DNN model
    h, w = 300, 300
    image_ndarray = cv2.resize(image_ndarray, (h, w))
    # Convert into blob
    blob = cv2.dnn.blobFromImage(image_ndarray, 1.0, (h, w), (104.0, 177.0, 123.0))
    # Convert into grayscale
    if grayscale:
        image_ndarray = cv2.cvtColor(image_ndarray, cv2.COLOR_BGR2GRAY)
    # Detect faces
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            # Get the coordinates of the bounding box
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x2, y2) = box.astype("int")

            # Extract the face ROI (region of interest) from the image
            face = image_ndarray[y:y2, x:x2]
            return cv2.resize(face, size), (x, y, x2, y2)
        else:
            return None, None


In [ ]:
# function to make the image size uniform and make train and test object with labels
class dataframe():
    def __init__(self):
        self.deva_faces = deva_faces
        self.swar_faces = swar_faces
        self.neg_faces = neg_faces
        self.images, self.labels = list(), list()

        self.total_images = len(deva_faces)+len(swar_faces)+len(neg_faces)
        self.labels_list = ['Unknown', 'Devasheesh', 'Swarnim']

        print('Total images: ', self.total_images)

        self.datagen = ImageDataGenerator(
            rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.2, # randomly shift images vertically (fraction of total height)
            shear_range=0.2,    # set range for random shear
            zoom_range=0.2,    # set range for random zoom
            horizontal_flip=True,   # randomly flip images
            fill_mode='nearest' # set mode for filling points outside the input boundaries
        )

    def append_dataframe(self, images_list, label, augment=False, grayscale=False):
        count = 0
        for image_adr in images_list:
            try:
                # read the image
                image = cv2.imread(image_adr)
                # extract the face
                face, _ = extract_frontal_face(image, grayscale=grayscale, size=(360, 480))

                print('Count: ', count, end='\r')
                if face is not None:
                    # print('Face type: ', type(face))
                    # print('Image shape: ', image.shape)
                    # print('Face shape: ', face.shape)
                    # print('Face: ', plt.imshow(face))

                    # append the face image to the list
                    if augment:
                        # using the image data generator to augment the images
                        i = 0
                        for batch in self.datagen.flow(face.reshape(1, 300, 300, 3), batch_size=1):
                            # append the image to the list
                            self.images.append(batch[0])
                            self.labels.append(label)
                            i += 1
                            if i > 10:
                                break
                        count += 1
                
                    else:
                        self.images.append(face)
                        self.labels.append(label)
                        count += 1
                else:
                    print(f'Face not found in {image_adr}')
                    continue
            except Exception as e:
                print(f'Error: {e} in {image_adr}')
                # logging
                # logging.error(f'Error: {e} in {image_adr}')
                continue
             
    def make_dataframe(self):
        self.append_dataframe(self.neg_faces, 0, augment=False)
        self.append_dataframe(self.deva_faces, 1, augment=True)
        self.append_dataframe(self.swar_faces, 2, augment=True)
        self.images = np.array(self.images)
        self.labels = np.array(self.labels)
        print('Images shape: ', self.images.shape)
        print('Labels shape: ', self.labels.shape)
        return self.images/255.0, self.labels, self.labels_list

x = dataframe()
images, labels, labels_list = x.make_dataframe()

In [ ]:
# saving numpy arrays
# np.save('images_v1.npy', images)
# np.save('labels_v1.npy', labels)

In [7]:
# loading numpy arrays
# labels_list = ['Unknown', 'Devasheesh', 'Swarnim']
# images = np.load('arrays/v1_pos_neg/images.npy')
# labels = np.load('arrays/v1_pos_neg/labels.npy')


In [ ]:
for x in range(700,705):
    try:
        plt.imshow(images[x])
        plt.title(labels_list[int(labels[x])])
        plt.show()
    except:
        print(f'Error in {x}')

## Splitting the data into train and test

In [ ]:
# split the data into train and test
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
images.shape

In [ ]:
train_images.shape

In [ ]:
train_labels.shape

In [ ]:
test_images.shape

In [ ]:
test_labels.shape

## Creating The CNN Model

In [ ]:
# Get the current physical devices
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Print the current device
    print("Currently using GPU:", gpus[0])
else:
    print("No GPU available.")


### Default working architecture (model_v1)

In [ ]:
model_v1 = models.Sequential()
model_v1.add(layers.Conv2D(
    64, (3, 3), activation='relu', input_shape=(360, 480, 3)))
model_v1.add(layers.MaxPooling2D((2, 2)))
model_v1.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model_v1.add(layers.MaxPooling2D((2, 2)))
model_v1.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model_v1.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
model_v1.add(layers.Flatten())
# model.add(layers.Dropout(0.5))
model_v1.add(layers.Dense(1024, activation='relu'))
model_v1.add(layers.Dense(512, activation='relu'))
model_v1.add(layers.Dense(256, activation='relu'))
model_v1.add(layers.Dense(128, activation='relu'))
model_v1.add(layers.Dense(64, activation='relu'))
model_v1.add(layers.Dense(32, activation='relu'))
model_v1.add(layers.Dense(len(labels_list), activation='softmax'))


In [ ]:
model_v1.summary()


In [ ]:
model_v1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model_v1.fit(train_images, train_labels,
                    epochs=20,
                    validation_data=(test_images, test_labels),
                    batch_size=16)


In [ ]:
test_loss, test_acc = model_v1.evaluate(test_images,  test_labels, verbose=2)
print('Accuracy:',test_acc)
print('Loss',test_loss)


In [ ]:
model_v1.save(
    './trained_models/face_recognition_model_v1_default_ep20_bs16.h5')


## v2: Adding more layers with more filters and proper structure

In [ ]:
def VGG_FACE_16_layers(input_shape, labels):
    model_vgg16 = tf.keras.models.Sequential()

    # block 1
    model_vgg16.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same',
              activation='relu', name='conv1_1', input_shape=input_shape))
    model_vgg16.add(tf.keras.layers.Conv2D(
        64, (3, 3), padding='same', activation='relu', name='conv1_2'))
    model_vgg16.add(tf.keras.layers.MaxPooling2D(
        (2, 2), strides=(2, 2), name='pool1'))

    # block 2
    model_vgg16.add(tf.keras.layers.Conv2D(
        128, (3, 3), padding='same', activation='relu', name='conv2_1'))
    model_vgg16.add(tf.keras.layers.Conv2D(
        128, (3, 3), padding='same', activation='relu', name='conv2_2'))
    model_vgg16.add(tf.keras.layers.MaxPooling2D(
        (2, 2), strides=(2, 2), name='pool2'))

    # block 3
    model_vgg16.add(tf.keras.layers.Conv2D(
        256, (3, 3), padding='same', activation='relu', name='conv3_1'))
    model_vgg16.add(tf.keras.layers.Conv2D(
        256, (3, 3), padding='same', activation='relu', name='conv3_2'))
    model_vgg16.add(tf.keras.layers.Conv2D(
        256, (3, 3), padding='same', activation='relu', name='conv3_3'))
    model_vgg16.add(tf.keras.layers.MaxPooling2D(
        (2, 2), strides=(2, 2), name='pool3'))

    # block 4
    model_vgg16.add(tf.keras.layers.Conv2D(
        512, (3, 3), padding='same', activation='relu', name='conv4_1'))
    model_vgg16.add(tf.keras.layers.Conv2D(
        512, (3, 3), padding='same', activation='relu', name='conv4_2'))
    model_vgg16.add(tf.keras.layers.Conv2D(
        512, (3, 3), padding='same', activation='relu', name='conv4_3'))
    model_vgg16.add(tf.keras.layers.MaxPooling2D(
        (2, 2), strides=(2, 2), name='pool4'))

    # block 5
    model_vgg16.add(tf.keras.layers.Conv2D(
        512, (3, 3), padding='same', activation='relu', name='conv5_1'))
    model_vgg16.add(tf.keras.layers.Conv2D(
        512, (3, 3), padding='same', activation='relu', name='conv5_2'))
    model_vgg16.add(tf.keras.layers.Conv2D(
        512, (3, 3), padding='same', activation='relu', name='conv5_3'))
    model_vgg16.add(tf.keras.layers.MaxPooling2D(
        (2, 2), strides=(2, 2), name='pool5'))

    # flatten and fully connected layers
    model_vgg16.add(tf.keras.layers.Flatten(name='flatten'))
    model_vgg16.add(tf.keras.layers.Dense(4096, activation='relu', name='fc6'))
    model_vgg16.add(tf.keras.layers.Dense(4096, activation='relu', name='fc7'))
    model_vgg16.add(tf.keras.layers.Dense(2622, activation='relu', name='fc8'))
    model_vgg16.add(tf.keras.layers.Dense(labels, activation='softmax'))

    return model_vgg16


model_vgg16 = VGG_FACE_16_layers((300, 300, 3), len(labels_list))


In [ ]:
model_vgg16.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_vgg16.summary()


In [ ]:
history = model_vgg16.fit(train_images, train_labels,
                    epochs=10,
                    validation_data=(test_images, test_labels),
                    batch_size=16)


In [ ]:
test_loss, test_acc = model_vgg16.evaluate(
    test_images,  test_labels, verbose=2)
print('Accuracy:',test_acc)
print('Loss',test_loss)


In [ ]:
model_vgg16.save('./trained_models/face_recognition_model_vgg16_ep10_bs16.h5')


## Testing the model

In [ ]:
# model.load_weights('face_recognition_model_ep2.h5')
# def predict(model, labels_list, grayscale=False):
#     # initialize the video capture object

#     # read the frame from the webcam
#     # frame = cv2.imread('data\Devasheesh\Devasheesh_32.jpg')
#     while True:
#         cap = cv2.VideoCapture(0)
#         ret, frame = cap.read()
#         frame = cv2.resize(frame, (300, 300))
#         # extract the face
#         face, (x, y, x2, y2) = extract_frontal_face(
#             frame, grayscale=grayscale, size=(300, 300))
#         if face is not None:
#             # predict the face
#             pred = model.predict(face.reshape(1, 300, 300, 3))
#             # get the label
#             label = labels_list[np.argmax(pred)]
#             # draw the rectangle and put the text
#             cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)
#             cv2.putText(frame, label, (x, y-10),
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

#         # show the frame
#         cv2.imshow('Face Recognition', frame)

# # call the predict function
# predict(model, labels_list, grayscale=False)
